In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os
import gc
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import keras
import keras.backend as K
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Conv1D, MaxPooling1D, GRU
from tensorflow.keras.models import Model, Sequential, load_model

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def decay(epochs):
    init = 1e-3
    drop = 10
    ratio = 0.9
    return max(5e-5, (init * (ratio ** (epochs//drop))))

es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
lrs = callbacks.LearningRateScheduler(decay, verbose=0)


In [ ]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import constraints

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
def build_crnn(shape):
    activation=mish
    input_ = Input(shape=shape)
    
    x = Conv1D(64, 10, strides=10, padding='same')(input_)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = MaxPooling1D(1)(x)
    
    x = Conv1D(128, 10, strides=10, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = MaxPooling1D(1)(x)
    
    x = Conv1D(256, 10, strides=10, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = MaxPooling1D(1)(x)

    x = GRU(256, return_sequences=True)(x)
    x = Attention(16)(x)
    
    x = Dense(256)(x)
    x = Activation(activation)(x)
    
    x = Dense(128)(x)
    x = Activation(activation)(x)
    
    x = Dropout(0.5)(x)
    output_ = Dense(30, activation='softmax')(x)
    
    model = Model(input_, output_)
    return model

In [ ]:
# Check List
## data shape: (n, ts_interval, n_cols)
## output type: clf, reconstruction
model = build_crnn((16000, 1))

In [ ]:
model.summary()